# Kaggle Titanic Challenge

Tratamento dos dados para o arquivo de teste e posterior formatação para o padrão de resposta esperado para o desafio.

In [1]:
# Configurações do notebook dentre outros
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, Markdown

# Imports gerais
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

# Imports específicos do Scikit-lean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [2]:
# Arquivos do Kaggle
url = "https://raw.githubusercontent.com/tekoryu/rough-on-rats/main/ml/data/titanic/test.csv"

# Carga no Pandas
df_teste = pd.read_csv(url, delimiter=',')

df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


## Pré Processamento

### Pronomes

In [3]:
def pega_pronome_tratamento(entrada):
    # A entrada deve ser um dataframe. Pega o Mr. Mrs. e coloca em coluna própria
    pronome_tratamento = re.search(r' ([A-Za-z]+)\.',entrada)
    if pronome_tratamento:
        return pronome_tratamento.group(1)
    return ""

df_teste['Pronome'] = df_teste['Name'].apply(pega_pronome_tratamento)

## Estoicismo

In [4]:
estoicismo = {
    'Capt': 5,
    'Col': 3,
    'Countess': 1,
    'Don': 5,
    'Dr': 3,
    'Jonkheer': 5,
    'Lady': 1,
    'Major': 3,
    'Master': 3,
    'Miss': 4,
    'Mlle': 1,
    'Mme': 1,
    'Mr': 2,
    'Mrs': 2,
    'Ms': 1,
    'Rev': 5,
    'Sir': 1
}
df_teste['Aval_Pronome'] = df_teste["Pronome"].apply(lambda x: estoicismo.get(x))

## Porto de Embarque

In [5]:
# Nao ha pelo visto sujeira nessa parte
df_teste["Embarked"].unique()

array(['Q', 'S', 'C'], dtype=object)

In [6]:
# Transforma o código do porto de embarque em escalar.
le = LabelEncoder()
df_teste["cod_embarque"] = le.fit_transform(df_teste['Embarked'])

## Valor do Ticket

In [7]:
df_teste["FixedFare"] = df_teste["Fare"]/(df_teste["SibSp"] + df_teste["Parch"] + 1)
df_teste["FixedFare"] = df_teste["FixedFare"].apply(np.round)

## Idade

In [8]:
# Pega a média das idades por pronome
pronomes = df_teste["Pronome"].unique()

# Pega a mediana por pronome
mediana = {}
for pronome in pronomes:
    mediana[pronome] = df_teste["Age"][df_teste["Pronome"] == pronome].median()

# Preenche as idades em branco com a media das idades para aquele pronome
df_teste['Age'] = df_teste.apply(lambda x: mediana[x['Pronome']] if pd.isna(x['Age']) else x['Age'], axis=1)

# se depois de preencher em branco nao melhorou
df_teste["Age"] = df_teste["Age"].fillna(df_teste["Age"].median())

# Converte para inteiro
df_teste["Age"] = df_teste["Age"].astype(int)

df_teste["classe_idade"] = pd.cut(df_teste["Age"], bins=[-np.inf,5,10,13,60,np.inf], labels=["Colo","Nova","Jovem","Adulto", "Idoso"])

## Categorias de idade

In [9]:
# Essa etapa pode ser suprimida, mas dificulta a legibilidade do caderno
cat_idades  = {
    'Colo': 1,
    'Nova': 2,
    'Jovem': 3,
    'Adulto': 4,
    'Idoso': 5,    
}

df_teste["classe_idade"] = df_teste["classe_idade"].map(cat_idades)

## Sexo

In [11]:
df_teste["Sex"] = df_teste["Sex"].map({'male':1,'female':0})

## Sozinho

In [12]:
df_teste["Sozinho"] = np.where(df_teste["SibSp"]+df_teste["Parch"]==0,1,0)

## Limpeza

In [ ]:
df_teste.drop(["PassengerId","Name","Ticket","Cabin","Embarked", "Pronome"], axis=1, inplace=True)


## Mudancas menores

In [14]:
# Converter moeda para inteiro pra ajudar na eficiência do SVM
df_teste["Fare"] = df_teste["Fare"].astype(int)
#df_treino["FixedFare"] = df_treino["FixedFare"].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [17]:
df_teste.isna().sum(axis=0).sort_values(ascending=False)

Cabin           327
Aval_Pronome      1
Fare              1
FixedFare         1
PassengerId       0
Age               0
Pclass            0
Sex               0
Name              0
Ticket            0
Parch             0
SibSp             0
Embarked          0
Pronome           0
cod_embarque      0
classe_idade      0
Sozinho           0
dtype: int64